In [5]:
import pandas as pd
import pickle
from pathlib import Path


In [6]:
import pickle
from pathlib import Path

with open(Path("../data/") / "match-transfermarkt-to-fbref" / "club_name_mapping_transfer_to_stat.pkl", "rb") as f:
    club_name_mapping = pickle.load(f)

In [7]:
club_name_mapping["AC Milan"] = "Milan"
club_name_mapping["AS Roma"] = "Roma"
club_name_mapping["Atalanta BC"] = "Atalanta"
club_name_mapping["Man City"] = "Manchester City"
club_name_mapping["Man Utd"] = "Manchester Utd"
club_name_mapping["Paris SG"] = "Paris S-G"
club_name_mapping["Sevilla FC"] = "Sevilla"
club_name_mapping["Leeds"] = "Leeds United"
club_name_mapping["Olympique Lyon"] = "Lyon"
club_name_mapping["FC Empoli'"] = "Empoli"
club_name_mapping["LOSC Lille"] = "Lille"
club_name_mapping["OGC Nice"] = "Nice"
club_name_mapping["E. Frankfurt"] = "Eint Frankfurt"
club_name_mapping["Real Betis"] = "Betis"
club_name_mapping["Celta de Vigo"] = "Celta Vigo"
club_name_mapping["FC Augsburg"] = "Augsburg"
club_name_mapping["Stade Rennais"] = "Rennes"
club_name_mapping["VfB Stuttgart"] = "Stuttgart"
club_name_mapping["Stade Reims"] = "Reims"
club_name_mapping["Bor. Dortmund"] = "Dortmund"
club_name_mapping["R. Strasbourg'"] = "Strasbourg"
club_name_mapping["G. Bordeaux"] = "Bordeaux"
club_name_mapping["Nottm Forest"] = "Nott'ham Forest"
club_name_mapping["TSG Hoffenheim"] = "Hoffenheim"
club_name_mapping["Leicester"] = "Leicester City"
club_name_mapping["B. Leverkusen"] = "Leverkusen"
club_name_mapping["Sheff Utd"] = "Sheffield Utd"
club_name_mapping["SC Freiburg"] = "Freiburg"
club_name_mapping["Bor. M'gladbach"] = "Gladbach"
club_name_mapping["Angers SCO"] = "Angers"
club_name_mapping["VfL Wolfsburg"] = "Wolfsburg"
club_name_mapping["FC Nantes"] = "Nantes"
club_name_mapping["Le Havre AC"] = "Le Havre"
club_name_mapping["FC Augsburg"] = "Augsburg"
club_name_mapping["AJ Auxerre"] = "Auxerre"
club_name_mapping["SSC Napoli"] = "Napoli"
club_name_mapping["R Charleroi SC"] = "Charleroi"
club_name_mapping["KRC Genk"] = "Genk"
club_name_mapping["RSC Anderlecht"] = "Anderlecht"
club_name_mapping["Royal Antwerp"] = "Antwerp"
club_name_mapping["KAA Gent"] = "Gent"
club_name_mapping["KVC Westerlo"] = "Westerlo"
club_name_mapping["Beerschot VA"] = "Beerschot"
club_name_mapping["Aalborg BK"] = "Aalborg"
club_name_mapping["FC Groningen"] = "Groningen"
club_name_mapping["FC Utrecht"] = "Utrecht"
club_name_mapping["Sparta R'dam"] = "Sparta R."
club_name_mapping["OFI Crete FC"] = "OFI Crete"
club_name_mapping["FC Porto"] = "Porto"
club_name_mapping["SC Braga"] = "Braga"
club_name_mapping["Aberdeen FC"] = "Aberdeen"
club_name_mapping["Hibernian FC"] = "Hibernian"
club_name_mapping["Kilmarnock FC"] = "Kilmarnock"



In [8]:
# write new club name mapping to file
with open(Path("../data/") / "match-transfermarkt-to-fbref" / "club_name_mapping_transfer_to_stat.pkl", "wb") as f:
    pickle.dump(club_name_mapping, f)

# also write to json
import json

with open(Path("../data/") / "match-transfermarkt-to-fbref" / "club_name_mapping_transfer_to_stat.json", "w") as f:
    json.dump(club_name_mapping, f)


In [10]:
df1 = pd.read_csv("../data/processed/test_match_logs_with_elos_20250528.csv")
df2 = pd.read_csv("../data/processed/final/test_match_logs_with_elos_20250606.csv")

In [11]:
len(df1), len(df2)

(40485, 59461)

In [18]:
df1["is_post_transfer"].sum(), df2["is_post_transfer"].sum()

(np.int64(24120), np.int64(35032))

In [22]:
df1[df1["is_post_transfer"]]["team_elo_from_club_elo"].notna().sum(), df2[df2["is_post_transfer"]]["team_elo_from_club_elo"].notna().sum()

(np.int64(23310), np.int64(33367))

In [39]:
df1[df1["team_elo_from_club_elo"].isna()].groupby("from_club").size().sort_values(ascending=False).head(20)

from_club
Real Sociedad      264
Spartak Moscow     150
Real Sociedad B    100
Köln                84
Saint-Étienne       74
Portimonense        71
Chaves              44
RB Leipzig          40
Benevento           37
1.FC Köln II        37
Greuther Fürth      33
Vitesse             33
Vizela              31
Dijon               29
1.FC Köln U19       25
TSG Hoffenheim      24
FC Copenhagen       22
Hannover 96         20
Union Berlin        20
1.FSV Mainz 05      20
dtype: int64

In [24]:
df2[df2["team_elo_from_club_elo"].isna()].groupby("from_club").size().sort_values(ascending=False)

from_club
Real Sociedad      264
Manchester Utd     216
Napoli             188
Spartak Moscow     150
Bordeaux           119
                  ... 
Nott'ham Forest      5
Newcastle            4
Dinamo Zagreb        4
CD Leganés           2
Preston              1
Length: 126, dtype: int64

In [27]:
import numpy as np

def calculate_elo_delta_feature(df: pd.DataFrame) -> pd.DataFrame:
    """
    Function to calculate the elo delta feature. This feature is the difference between
    the current team elo for post-transfer matches and the last team elo from the club elo
    prior to the transfer.
    """
    
    df = df.copy()
    df['delta_elo_post_transfer'] = np.nan
    
    # Group by transfer_id to process each transfer separately
    for transfer_id, transfer_group in df.groupby('transfer_id'):
        transfer_group_sorted = transfer_group.sort_values('date')
        pre_transfer_rows = transfer_group_sorted[
            (transfer_group_sorted['is_pre_transfer']) & 
            (transfer_group_sorted['team_elo_from_club_elo'].notna())
        ]
        if len(pre_transfer_rows) > 0:
            # Get the last pre-transfer elo value (chronologically last)
            last_pre_transfer_elo = pre_transfer_rows['team_elo_from_club_elo'].iloc[-1]
            # Calculate delta for post-transfer rows with non-null elo values
            post_transfer_mask = (
                (df['transfer_id'] == transfer_id) & 
                (df['is_post_transfer']) &
                (df['team_elo_from_club_elo'].notna())
            )
            df.loc[post_transfer_mask, 'delta_elo_post_transfer'] = (
                df.loc[post_transfer_mask, 'team_elo_from_club_elo'] - last_pre_transfer_elo
            )
    return df

In [29]:
df1 = calculate_elo_delta_feature(df1)
df2 = calculate_elo_delta_feature(df2)

(np.int64(33845), np.int64(49984))

In [35]:
df1["is_post_transfer"].sum(), df2["delta_elo_post_transfer"].notna().sum()


(np.int64(24120), np.int64(9477))

In [38]:
len(df1[df1["is_post_transfer"] & df1["delta_elo_post_transfer"].notna()]), len(df2[df2["is_post_transfer"] & df2["delta_elo_post_transfer"].notna()])

(6640, 9477)

In [31]:
df2["delta_elo_post_transfer"].describe()

count    9477.000000
mean       14.467703
std       142.421475
min      -384.843262
25%       -81.666626
50%        16.096069
75%       111.599854
max       485.055054
Name: delta_elo_post_transfer, dtype: float64

In [43]:
df1[df1["is_post_transfer"] & df1["delta_elo_post_transfer"].isna()].groupby("from_club").size().sort_values(ascending=False).head(20)

from_club
Atalanta BC        392
AS Roma            330
Man City           287
AC Milan           255
Without Club       220
Man Utd            196
Inter              190
Olympique Lyon     189
Real Sociedad      178
Real Valladolid    177
Paris SG           171
FC Empoli          170
VfL Wolfsburg      160
Juventus           160
Sevilla FC         159
Stade Rennais      143
Atlético Madrid    140
Watford            139
LOSC Lille         128
Juve Next Gen      127
dtype: int64

In [44]:
df2[df2["is_post_transfer"] & df2["delta_elo_post_transfer"].isna()].groupby("from_club").size().sort_values(ascending=False).head(20)

from_club
Without Club       376
Atalanta           304
Manchester Utd     282
Chelsea            272
Manchester City    263
Roma               251
Rennes             233
Inter              230
Napoli             228
Arsenal            220
Real Sociedad      218
RB Leipzig         210
Juventus           210
Paris S-G          203
Tottenham          191
FC Empoli          190
Milan              186
Atlético Madrid    180
Real Valladolid    177
Barcelona          177
dtype: int64

In [51]:
df2[df2["is_post_transfer"] & df2["delta_elo_post_transfer"].isna() & (df2["from_club"] == "Paris S-G")][
    ["from_club", "to_club", "date", "date", "team", "opponent", "position", "is_post_transfer", "team_elo_from_club_elo", "delta_elo_post_transfer"]
]

,from_club,to_club,date,date,team,opponent,position,is_post_transfer,team_elo_from_club_elo,delta_elo_post_transfer
3338,Paris S-G,West Ham,2021-08-15,2021-08-15,West Ham,Newcastle Utd,NaN,True,1810.996338,NaN
3339,Paris S-G,West Ham,2021-08-23,2021-08-23,West Ham,Leicester City,NaN,True,1819.103516,NaN
3340,Paris S-G,West Ham,2021-08-28,2021-08-28,West Ham,Crystal Palace,NaN,True,1832.529663,NaN
3341,Paris S-G,West Ham,2021-09-11,2021-09-11,West Ham,Southampton,NaN,True,1828.108276,NaN
3342,Paris S-G,West Ham,2021-09-19,2021-09-19,West Ham,Manchester Utd,NaN,True,1832.212524,NaN
...,...,...,...,...,...,...,...,...,...,...
58206,Paris S-G,RB Leipzig,2025-02-14,2025-02-14,RB Leipzig,Augsburg,AM,True,1747.827148,NaN
58207,Paris S-G,RB Leipzig,2025-02-23,2025-02-23,RB Leipzig,Heidenheim,AM,True,1740.722778,NaN
58208,Paris S-G,RB Leipzig,2025-03-01,2025-03-01,RB Leipzig,Mainz 05,AM,True,1734.353271,NaN
58209,Paris S-G,RB Leipzig,2025-03-08,2025-03-08,RB Leipzig,Freiburg,LM,True,1726.180786,NaN


In [33]:
df1["delta_elo_post_transfer"].notna().sum(), df2["delta_elo_post_transfer"].notna().sum()


(np.int64(6640), np.int64(9477))

In [9]:
DATA_PATH = Path("../data")

In [28]:
df_players = pd.read_csv(DATA_PATH / "processed" / "players_to_scrape_20250423.csv", sep=",")

In [29]:
df_players

,player_name,player_id,transfer_id,transfer_date,transfer_season,from_club,to_club,from_club_name_transfermarkt,to_club_name_transfermarkt,league,from_club_domestic_competition_id,from_club_domestic_competition_name,to_club_domestic_competition_id,to_club_domestic_competition_name
0,Aaron Hickey,Aaron-Hickey,Aaron Hickey_Bologna_Brentford_2022-07-09,2022-07-09,22/23,Bologna,Brentford,Bologna,Brentford,EPL,IT1,SerieA,GB1,PremierLeague
1,Aaron Ramsdale,Aaron-Ramsdale,Aaron Ramsdale_Sheff Utd_Arsenal_2021-08-20,2021-08-20,21/22,Sheff Utd,Arsenal,Sheff Utd,Arsenal,EPL,GB1,PremierLeague,GB1,PremierLeague
2,Aaron Ramsdale,Aaron-Ramsdale,Aaron Ramsdale_Arsenal_Southampton_2024-08-30,2024-08-30,24/25,Arsenal,Southampton,Arsenal,Southampton,EPL,GB1,PremierLeague,GB1,PremierLeague
3,Aaron Wan-Bissaka,Aaron-Wan-Bissaka,Aaron Wan-Bissaka_Man Utd_West Ham_2024-08-13,2024-08-13,24/25,Man Utd,West Ham,Man Utd,West Ham,EPL,GB1,PremierLeague,GB1,PremierLeague
4,Aarón Martín,Aaron-Martin,Aarón Martín_1.FSV Mainz 05_Genoa_2023-07-03,2023-07-03,23/24,1.FSV Mainz 05,Genoa,1.FSV Mainz 05,Genoa,SerieA,L1,Bundesliga,IT1,SerieA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1892,Çağlar Söyüncü,Caglar-Soyuncu,Çağlar Söyüncü_Leicester_Atlético Madrid_2023-...,2023-07-05,23/24,Leicester,Atlético Madrid,Leicester,Atlético Madrid,LaLiga,GB1,PremierLeague,ES1,LaLiga
1893,Óscar Melendo,Oscar-Melendo,Óscar Melendo_Espanyol_Granada_2022-08-06,2022-08-06,22/23,Espanyol,Granada,Espanyol,Granada CF,LaLiga,ES1,LaLiga,ES1,LaLiga
1894,Óscar Rodríguez Arnaiz,Oscar-Rodriguez-Arnaiz,Óscar Rodríguez Arnaiz_Sevilla FC_Getafe_2022-...,2022-01-17,21/22,Sevilla FC,Getafe,Sevilla FC,Getafe,LaLiga,ES1,LaLiga,ES1,LaLiga
1895,Óscar Rodríguez Arnaiz,Oscar-Rodriguez-Arnaiz,Óscar Rodríguez Arnaiz_Sevilla FC_Getafe_2023-...,2023-09-01,23/24,Sevilla FC,Getafe,Sevilla FC,Getafe,LaLiga,ES1,LaLiga,ES1,LaLiga


In [31]:
df_names = df_players[
    ["from_club_name_transfermarkt",
     "to_club_name_transfermarkt",
     "from_club",
     "to_club",
     "from_club_domestic_competition_id",
     "to_club_domestic_competition_id",
     "from_club_domestic_competition_name",
     "to_club_domestic_competition_name"]
]

In [33]:
df_names.rename(columns={
    "from_club": "from_club_fbref",
    "to_club": "to_club_fbref",
    "from_club_domestic_competition_id": "from_club_domestic_competition_id_fbref",
    "to_club_domestic_competition_id": "to_club_domestic_competition_id_fbref",
}, inplace=True)

/var/folders/lj/gw7wtc1n5d52d9p2hmd4rqdc0000gn/T/ipykernel_45373/4151660500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_names.rename(columns={


,from_club_name_transfermarkt,to_club_name_transfermarkt,from_club_fbref,to_club_fbref,from_club_domestic_competition_id_fbref,to_club_domestic_competition_id_fbref,from_club_domestic_competition_name,to_club_domestic_competition_name
0,Bologna,Brentford,Bologna,Brentford,IT1,GB1,SerieA,PremierLeague
1,Sheff Utd,Arsenal,Sheff Utd,Arsenal,GB1,GB1,PremierLeague,PremierLeague
2,Arsenal,Southampton,Arsenal,Southampton,GB1,GB1,PremierLeague,PremierLeague
3,Man Utd,West Ham,Man Utd,West Ham,GB1,GB1,PremierLeague,PremierLeague
4,1.FSV Mainz 05,Genoa,1.FSV Mainz 05,Genoa,L1,IT1,Bundesliga,SerieA


In [34]:
df_names.head()

,from_club_name_transfermarkt,to_club_name_transfermarkt,from_club_fbref,to_club_fbref,from_club_domestic_competition_id_fbref,to_club_domestic_competition_id_fbref,from_club_domestic_competition_name,to_club_domestic_competition_name
0,Bologna,Brentford,Bologna,Brentford,IT1,GB1,SerieA,PremierLeague
1,Sheff Utd,Arsenal,Sheff Utd,Arsenal,GB1,GB1,PremierLeague,PremierLeague
2,Arsenal,Southampton,Arsenal,Southampton,GB1,GB1,PremierLeague,PremierLeague
3,Man Utd,West Ham,Man Utd,West Ham,GB1,GB1,PremierLeague,PremierLeague
4,1.FSV Mainz 05,Genoa,1.FSV Mainz 05,Genoa,L1,IT1,Bundesliga,SerieA


In [35]:
### Write code to have a unique file with the following columns:
# club_name_transfermarkt, club_name_fbref, club_domestic_competition_id_transfermarkt, club_domestic_competition_name_fbref

